In [1]:
import pandas as pd
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python')

ce5.head()

,0,1,2,3,4,5
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."


In [2]:
ce5_mod = ce5[ce5[5].str.contains('/usr/bin/sbatch')].reset_index(drop=True)

In [3]:
ce5_mod.isna().sum()

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

In [4]:
ce5_mod.duplicated().sum()

0

In [5]:
ce5_mod

,0,1,2,3,4,5
0,2020-10-16 08:32:25.162144,user 9201,retry 0,time 0.12207937240600586,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
1,2020-10-16 08:32:25.254135,user 9201,retry 0,time 0.12226557731628418,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
2,2020-10-16 08:32:53.021570,user 9201,retry 0,time 0.07345795631408691,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
3,2020-10-16 08:32:53.063351,user 9201,retry 0,time 0.06784367561340332,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
4,2020-10-16 08:32:53.263876,user 9201,retry 0,time 0.19620347023010254,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
...,...,...,...,...,...,...
910529,2021-10-07 21:44:59.008524,user 9204,retry 0,time 0.029953718185424805,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
910530,2021-10-07 21:53:47.800229,user 9204,retry 0,time 0.03073716163635254,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
910531,2021-10-07 21:54:51.314349,user 9203,retry 0,time 0.028333425521850586,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
910532,2021-10-07 21:55:01.270660,user 9203,retry 0,time 0.03251481056213379,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."


In [6]:
#looking for specific user with error code
ce5_mod[(ce5_mod[1] == 'user 9201') & (ce5_mod[4] == 'returncode 255')]

,0,1,2,3,4,5
35195,2020-10-21 18:57:44.868978,user 9201,retry 1,time 0.014236211776733398,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
35196,2020-10-21 18:57:44.882731,user 9201,retry 2,time 0.013496637344360352,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
35198,2020-10-21 18:57:44.947157,user 9201,retry 1,time 0.014467954635620117,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
35199,2020-10-21 18:57:44.960924,user 9201,retry 2,time 0.013517141342163086,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
35203,2020-10-21 18:57:44.993939,user 9201,retry 1,time 0.016413450241088867,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
...,...,...,...,...,...,...
513242,2021-05-13 20:59:38.368666,user 9201,retry 0,time 0.06008148193359375,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
839300,2021-08-31 10:08:48.013014,user 9201,retry 0,time 0.39307546615600586,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
856101,2021-09-09 03:45:41.333430,user 9201,retry 0,time 0.07905292510986328,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
868918,2021-09-15 16:10:53.239301,user 9201,retry 0,time 0.04546403884887695,returncode 255,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."


In [7]:
#lok user 9204
ce5_mod[(ce5_mod[1] == 'user 9204') & (ce5_mod[4] == 'returncode 1')].head(25)

,0,1,2,3,4,5
13884,2020-10-18 06:53:44.272915,user 9204,retry 0,time 20.038464307785034,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
13885,2020-10-18 06:54:04.322412,user 9204,retry 1,time 20.048906326293945,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
13950,2020-10-18 07:47:25.825172,user 9204,retry 0,time 20.082627773284912,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
13954,2020-10-18 07:47:45.871008,user 9204,retry 1,time 20.045220851898193,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
13980,2020-10-18 07:53:33.972840,user 9204,retry 0,time 20.04148578643799,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
14916,2020-10-18 16:02:01.338468,user 9204,retry 0,time 20.03518772125244,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
16037,2020-10-18 20:52:15.737852,user 9204,retry 0,time 20.064260244369507,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
16672,2020-10-19 00:23:37.945125,user 9204,retry 0,time 20.037097215652466,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
16673,2020-10-19 00:23:57.979047,user 9204,retry 1,time 20.03341293334961,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
16735,2020-10-19 01:01:02.211847,user 9204,retry 0,time 20.038537979125977,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scr..."
